In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import pickle
from datetime import datetime, timedelta
import warnings
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import os
import gc
import time
from typing import Dict, List, Tuple, Optional

warnings.filterwarnings('ignore')

In [4]:
def setup_rtx4050():
    """Setup RTX 4050 with proper CUDA detection."""
    # Check if PyTorch can see CUDA
    cuda_available = torch.cuda.is_available()
    
    if cuda_available:
        device = torch.device('cuda')
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        cuda_version = torch.version.cuda
        
        print(f"✅ GPU: {gpu_name}")
        print(f"✅ Memory: {gpu_memory:.1f} GB")
        print(f"✅ CUDA: {cuda_version}")
        print(f"✅ PyTorch: {torch.__version__}")
        
        # Test GPU computation
        try:
            x = torch.randn(1000, 1000, device='cuda')
            y = torch.mm(x, x)
            torch.cuda.synchronize()
            print(f"✅ GPU computation: Working")
            
            # Set memory fraction for RTX 4050
            torch.cuda.set_per_process_memory_fraction(0.9)  # Use 90% of VRAM
            
            return device, True, gpu_memory
        except Exception as e:
            print(f"❌ GPU computation failed: {e}")
            return torch.device('cpu'), False, 0
    else:
        print(f"❌ CUDA not available")
        print(f"🔧 Fix with: pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121")
        return torch.device('cpu'), False, 0

device, gpu_working, gpu_memory = setup_rtx4050()

# RTX 4050 optimized config
if gpu_working:
    CONFIG = {
        'batch_size': 128,          # Optimal for 6GB RTX 4050
        'sequence_length': 60,      # 60 days lookback
        'hidden_size': 128,         # Good balance for RTX 4050
        'num_attention_heads': 8,
        'num_layers': 3,
        'dropout': 0.1,
        'learning_rate': 0.001,
        'epochs': 50,              # Real training epochs
        'early_stopping_patience': 10,
        'mixed_precision': True,    # FP16 for RTX 4050
        'grad_clip': 1.0,
        'device': device
    }
    print(f"🎮 RTX 4050 optimized config loaded")
else:
    CONFIG = {
        'batch_size': 32,
        'sequence_length': 30,
        'hidden_size': 64,
        'num_attention_heads': 4,
        'num_layers': 2,
        'dropout': 0.1,
        'learning_rate': 0.001,
        'epochs': 20,
        'early_stopping_patience': 5,
        'mixed_precision': False,
        'grad_clip': 1.0,
        'device': device
    }
    print(f"💻 CPU fallback config loaded")

print(f"⚙️ Config: {CONFIG}")

❌ CUDA not available
🔧 Fix with: pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
💻 CPU fallback config loaded
⚙️ Config: {'batch_size': 32, 'sequence_length': 30, 'hidden_size': 64, 'num_attention_heads': 4, 'num_layers': 2, 'dropout': 0.1, 'learning_rate': 0.001, 'epochs': 20, 'early_stopping_patience': 5, 'mixed_precision': False, 'grad_clip': 1.0, 'device': device(type='cpu')}
